### EN: Overview of Retailrocket events data
### FA: نمای کلی داده‌های رویداد Retailrocket

In [1]:
# EN: Imports and plotting configuration
# FA: بارگذاری کتابخانه‌ها و تنظیمات ترسیم
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

plt.style.use("seaborn-v0_8-whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["axes.titlesize"] = 14
plt.rcParams["axes.labelsize"] = 12

### EN: Load processed datasets
### FA: بارگذاری داده‌های پردازش‌شده

In [2]:
# EN: Adjust the paths if running from a different working directory
# FA: در صورت اجرای نوت‌بوک از مسیر دیگر، مسیرها را تنظیم کنید
events_path = "../data/processed/events_clean.parquet"
matrix_path = "../data/processed/user_item_interactions.npz"

events = pd.read_parquet(events_path)

# EN: Interaction matrix is optional for quick stats
# FA: ماتریس تعاملی برای آمار سریع اختیاری است
try:
    from scipy import sparse
    interaction_matrix = sparse.load_npz(matrix_path)
except FileNotFoundError:
    interaction_matrix = None
    print("Interaction matrix not found; run preprocessing first.")

### EN: Quick data preview
### FA: پیش‌نمایش سریع داده‌ها

In [3]:
# EN: Inspect head and basic info
# FA: بررسی سرستون‌ها و اطلاعات پایه
display(events.head())
display(events.info())
display(events.describe(include="all"))

### EN: Distribution of event types
### FA: توزیع انواع رویداد

In [4]:
# EN: Count how frequent each event type is
# FA: فراوانی هر نوع رویداد را محاسبه می‌کنیم
event_counts = events["event"].value_counts()
sns.barplot(x=event_counts.index, y=event_counts.values)
plt.title("Event Type Frequency")
plt.xlabel("Event Type")
plt.ylabel("Count")
plt.show()

### EN: Interactions per user
### FA: تعداد تعاملات به ازای هر کاربر

In [5]:
# EN: Calculate how many events each user has
# FA: تعداد رویدادهای هر کاربر را محاسبه می‌کنیم
user_event_counts = events.groupby("visitorid")['event'].count()
sns.histplot(user_event_counts, bins=50, log_scale=(True, False))
plt.title("Events per User (log-scaled bins)")
plt.xlabel("Events per User")
plt.ylabel("Frequency")
plt.show()

### EN: Interactions per item
### FA: تعداد تعاملات به ازای هر کالا

In [6]:
# EN: Calculate how many events each item has
# FA: تعداد رویدادهای هر کالا را محاسبه می‌کنیم
item_event_counts = events.groupby("itemid")['event'].count()
sns.histplot(item_event_counts, bins=50, log_scale=(True, False))
plt.title("Events per Item (log-scaled bins)")
plt.xlabel("Events per Item")
plt.ylabel("Frequency")
plt.show()

### EN: Time range of events
### FA: بازه زمانی رویدادها

In [7]:
# EN: Calculate min and max timestamps
# FA: کمینه و بیشینه زمان رویدادها را محاسبه می‌کنیم
min_time = events['timestamp'].min()
max_time = events['timestamp'].max()
print(f"Time span: {min_time} to {max_time}")

### EN: Interaction matrix snapshot (if available)
### FA: نمایی از ماتریس تعاملی (در صورت موجود بودن)

In [8]:
if interaction_matrix is not None:
    # EN: Basic stats on sparsity
    # FA: آمار اولیه از تنکی ماتریس
    num_users, num_items = interaction_matrix.shape
    nnz = interaction_matrix.nnz
    sparsity = 1 - (nnz / (num_users * num_items))
    print(f"Users: {num_users:,} | Items: {num_items:,} | Non-zeros: {nnz:,}")
    print(f"Sparsity: {sparsity:.4f}")
else:
    print("Interaction matrix not loaded.")